In [2]:
import json
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [3]:
model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

# 2️⃣ Savollarni yuklash
with open("mentor_questions.json", "r", encoding="utf-8") as f:
    data = json.load(f)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [4]:
questions = [item["question"] for item in data]

# 3️⃣ Embedding
embeddings = model.encode(questions)

In [7]:
similarity_matrix = cosine_similarity(embeddings)

# 5️⃣ Guruhlash
THRESHOLD = 0.68  # 0.7–0.8 ideal
groups = []
used = set()

In [8]:
for i in range(len(questions)):
    if i in used:
        continue

    group = [questions[i]]
    used.add(i)

    for j in range(i + 1, len(questions)):
        if similarity_matrix[i][j] >= THRESHOLD:
            group.append(questions[j])
            used.add(j)

    groups.append(group)

print(f"✅ {len(groups)} ta savol guruhi topildi")

✅ 547 ta savol guruhi topildi


In [9]:
from sklearn.cluster import DBSCAN

clustering = DBSCAN(
    eps=0.32,          # 0.3–0.4 oralig‘ini sinab ko‘r
    min_samples=2,
    metric="cosine"
).fit(embeddings)

clusters = {}

for idx, label in enumerate(clustering.labels_):
    if label == -1:
        continue
    clusters.setdefault(label, []).append(questions[idx])

print(f"📊 Guruhlar soni: {len(clusters)}")


📊 Guruhlar soni: 21


In [15]:
labels = clustering.labels_

total = len(labels)
noise = list(labels).count(-1)
clusters = len(set(labels)) - (1 if -1 in labels else 0)

print("Jami savollar:", total)
print("Noise (-1):", noise)
print("Real guruhlar:", clusters)


Jami savollar: 1333
Noise (-1): 234
Real guruhlar: 21


In [11]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def get_representative_question(group_questions, model):
    """
    Guruh ichidan eng markaziy savolni topadi
    """
    embs = model.encode(group_questions)
    sim = cosine_similarity(embs)

    avg_sim = sim.mean(axis=1)
    best_idx = avg_sim.argmax()

    return group_questions[best_idx]


In [12]:
pairwise_faq = []

for idx, group in enumerate(groups, start=1):
    main_q = get_representative_question(group, model)

    pairwise_faq.append({
        "faq_id": idx,
        "question": main_q,
        "variants": group
    })

print("Pairwise FAQ soni:", len(pairwise_faq))


Pairwise FAQ soni: 547


In [13]:
import json

with open("faq_pairwise.json", "w", encoding="utf-8") as f:
    json.dump(pairwise_faq, f, ensure_ascii=False, indent=2)


In [17]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(
    eps=0.32,          # keyin o‘zgartirib ko‘rishing mumkin
    min_samples=2,
    metric="cosine"
)

dbscan.fit(embeddings)

labels = dbscan.labels_

clusters = {}  # MUHIM: dict bo‘lishi shart

for idx, label in enumerate(labels):
    if label == -1:
        continue  # noise tashlab yuboramiz

    clusters.setdefault(label, []).append(questions[idx])

print("DBSCAN guruhlar soni:", len(clusters))


DBSCAN guruhlar soni: 21


In [21]:
dbscan_faq = []

for label, group in clusters.items():
    main_q = get_representative_question(group, model)

    dbscan_faq.append({
    "cluster_id": int(label),   # 🔥 MUHIM
        "question": main_q,
        "variants": group
    })

print("DBSCAN FAQ:", len(dbscan_faq))


DBSCAN FAQ: 21


In [22]:
import json

with open("faq_top_dbscan.json", "w", encoding="utf-8") as f:
    json.dump(dbscan_faq, f, ensure_ascii=False, indent=2)

print("✅ faq_top_dbscan.json saqlandi")


✅ faq_top_dbscan.json saqlandi
